In [133]:
# Python script to initialise trajectories on a neutral density surface, for use with Ariane
# GM 23/9/16: Presently, this script places one particle per grid cell, suitable for many applications. It will need to be adapted if I wish to initialise particles at higher resolution.

In [134]:
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import os

In [135]:
# TRAJECTORY INITIALISATION OPTIONS
# Target density surface
sigT = 27.8
# Times of initialisation
time = np.array([4107, 4180])

In [136]:
# ARIANE EXPERIMENT DIRECTORY
arianedir = '/home/ocean2/graemem/ariane/experiments/orca025_global_5d/'
experiment = 'quant_back_seedGLOBALn1_tX_sign10_MLrefz8delsig0.01'

In [137]:
# DATASET LOCATIONS
# Specify the locations and filenames of the density and grid data
rootdir = '/home/ocean2/graemem/ariane/'
datadir = 'data_link/orca025_global_5d/'
griddir = 'grids_ncmod/'
gridfile = 'ORCA025.L75-GRD88_mesh_hgr.nc'

# LOAD GRID DATA
grid = Dataset(rootdir+griddir+gridfile,'r')
deptht = grid.variables['gdept_0']
e1t = grid.variables['e1t']
e2t = grid.variables['e2t']

In [142]:
ntot=0 # Keep track of number of initialised particles
# INITIALISE TIME LOOP
for t in time:
    # TIME DEPENDENT DATA FILES
    datafileR = 'ORCA025.L75-GJM189_'+str(t)+'_gridRn.nc' # Density data
    datafileT = 'ORCA025.L75-GJM189_'+str(t)+'_gridT.nc' # Temperature and salinity data
    print datafileR

    # LOAD DENSITY
    dataR=Dataset(rootdir+datadir+datafileR,'r')
    vosigntr=dataR.variables['vosigntr']
    # Extract dimensions
    (nz,ny,nx)=vosigntr.shape[1:4]

    # LOAD TEMPERATURE AND SALINITY
    dataT = Dataset(rootdir+datadir+datafileT,'r')
    votemper=dataT.variables['votemper']
    vosaline=dataT.variables['vosaline']

    # FIND THE VERTICAL INDEX OF A TARGET DENSITY SURFACE AT EACH POINT IN SPACE
    # In addition, vertically interpolate tracer values (temperature, salinity) onto this surface (required input for Ariane)
    k = np.empty((nx,ny,))
    k.fill(np.nan)
    t = np.empty((nx,ny,))
    t.fill(np.nan)
    s = np.empty((nx,ny,))
    s.fill(np.nan)
    for i in range(0,nx):
        #print(i)
        for j in range(0,ny):
            # Find where the column density is less than and greater than the target density
            lessarray = np.where(vosigntr[0,:,j,i]-sigT < 0)[0]
            morearray = np.where(vosigntr[0,:,j,i]-sigT > 0)[0]
            if morearray.size==0 or lessarray.size==0: # The surface does not exist
                k[i,j]=np.nan
            else:
                less = lessarray[-1] # Last depth index where the density is less than the target density
                more = morearray[0] # First depth index where the density is greater than the target density
                # 'less' index should be smaller than 'more' index (for a stable water column). If this is not the case, skip past this grid cell.
                if less<more:
                    # Determine how far target surface is from the 'less' surface
                    diffless = abs(vosigntr[0,less,j,i]-sigT)
                    diffmore = abs(vosigntr[0,more,j,i]-sigT)
                    ratio = diffless/np.sum([diffless, diffmore]) # Distance from 'less' surface as a ratio of total density difference
                    # Specify vertical index for initial particle position
                    k[i,j] = less+ratio+0.5 # Addition of 0.5 adjusts for Ariane indexing conventions (indices are relative to velocity points)
                    # Interpolate tracer values
                    t[i,j] = votemper[0,less,j,i]+ratio*np.absolute(votemper[0,more,j,i]-votemper[0,less,j,i])
                    s[i,j] = vosaline[0,less,j,i]+ratio*np.absolute(vosaline[0,more,j,i]-vosaline[0,less,j,i])
                else:
                    k[i,j]=np.nan

    # PUT INDICES INTO ARIANE FORMAT
    # Specify arrays
    init_x = np.array([])
    init_y = np.array([])
    init_z = np.array([])
    init_temp = np.array([])
    init_salt = np.array([])
    init_dens = np.array([])
    init_area= np.array([])
    init_transp = np.array([])

    for i in range(500,510):
        for j in range(500,510):
            if np.isfinite(k[i,j]):
                # For positioning: -0.5 due to Ariane convention (index relative to velocity points), +1 due to python indexing conventions
                init_x = np.append(init_x,i-0.5+1)
                init_y = np.append(init_y,j-0.5+1)
                init_z = np.append(init_z,k[i,j])
                init_temp = np.append(init_temp,t[i,j])
                init_salt = np.append(init_salt,s[i,j])
                init_dens = np.append(init_dens,sigT)
                init_area = np.append(init_area,e1t[0,j,i]*e2t[0,j,i])
                init_transp = np.append(init_transp,np.nan) # GM 23/9/16: Presently, do not specify a transport. Fudge to satisfy Ariane.

                # STEP FORWARD NTOT
                ntot = ntot+1



ORCA025.L75-GJM189_4107_gridRn.nc


KeyboardInterrupt: 

In [132]:
# SAVE AS A NETCDF FILE
ariane = Dataset(arianedir+experiment+'/ariane_initial.nc','w',format='NETCDF4_CLASSIC')
# Create dimensions
ariane.createDimension('ntot',ntot)
# Create variables
x = ariane.createVariable('init_x','f8',('ntot',))
y = ariane.createVariable('init_y','f8',('ntot',))
z = ariane.createVariable('init_z','f8',('ntot',))
temp = ariane.createVariable('init_temp','f8',('ntot',))
salt = ariane.createVariable('init_salt','f8',('ntot',))
dens = ariane.createVariable('init_dens','f8',('ntot',))
area = ariane.createVariable('init_area','f8',('ntot',))
transp = ariane.createVariable('init_transp','f8',('ntot',))
# Write to variables
x = init_x
y = init_y
z = init_z
temp = init_temp
salt = init_salt
dens = init_dens
area = init_area
transp = init_transp
# Close netcdf
ariane.close()

In [144]:
print j

659
